# Análise: Microdados ENEM 2022 - Introdução

O presente projeto tem como objetivo realizar uma análise detalhada dos microdados do Exame Nacional do Ensino Médio (ENEM) referentes ao ano de 2022. A análise abordará não apenas os resultados obtidos pelos participantes, mas também explorará as relações existentes entre esses resultados e os dados de inscrição, como sexo declarado, idade, cor/raça, estado, região, tipo de escola, localização do município, entre outros. Além disso, serão consideradas as respostas obtidas no questionário socioeconômico, permitindo uma análise abrangente dos fatores socioeconômicos relacionados ao desempenho dos participantes no exame.

O projeto será dividido em quatro etapas principais:

1.) Importação dos dados: Nesta etapa, os microdados do ENEM 2022 serão importados e organizados para análise. Será utilizada uma abordagem adequada para a extração dos dados necessários, garantindo a integridade e qualidade dos mesmos (https://download.inep.gov.br/microdados/microdados_enem_2022.zip);

2.) Tratamento dos dados: Uma vez que os dados forem importados, será realizada uma minuciosa etapa de tratamento, onde serão tratadas questões como valores ausentes, inconsistências e erros de formatação. Essa etapa é essencial para assegurar a confiabilidade e a consistência dos dados a serem analisados posteriormente;

3.) Análise exploratória dos dados: Nesta etapa, será realizada uma análise abrangente e exploratória dos dados, visando identificar padrões, tendências e insights relevantes. Serão utilizadas técnicas estatísticas e ferramentas de visualização de dados para auxiliar na compreensão e interpretação dos resultados; e

4.) Criação de dashboard interativo: Por fim, os resultados obtidos serão apresentados de forma clara e acessível através da criação de um dashboard interativo. Essa ferramenta permitirá uma visualização dinâmica e personalizada dos dados, facilitando a exploração dos resultados e possibilitando uma melhor compreensão das relações entre as variáveis analisadas.

# Importação de bibliotecas e dados

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import plotly.express as px

pd.set_option('display.max_columns', None)

In [2]:
dados = pd.read_csv('MICRODADOS_ENEM_2022.csv', sep=';', encoding='latin-1')

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476105 entries, 0 to 3476104
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

O Enem aceita inscrições de alunos que ainda não concluíram o ensino médio no ano de realização do certame. Para estes alunos é atribuída a condição de 'Treineiro'. A coluna "IN_TREINEIRO" mostra valor 1 para os cadidatos que satisfazem essa condição e 0 para os demais.

Como o conjunto de dados é grande e usa muita memória (memory usage: 2.0+ GB), optei por selecionar apenas os candidatos não treineiros, o que não será capaz de prejudicar a análise.

In [4]:
df_1 = dados.loc[dados['IN_TREINEIRO'] == 0]

Após abrir o arquivo "Dicionário_Microdados_Enem_2022.xlsx" (disponível com o arquivo .zip baixado) e todo o conteúdo dos campos da tabela completa, optei por descartar as informações abaixo. Serão descartadas também as linhas que contém valores NaN, uma vez que isso ocorre nos casos em que o candidato faltou pelo menos uma prova, não interessando para esta análise. 

In [5]:
df_1 = df_1.drop(columns = ['NU_ANO', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO','TP_ANO_CONCLUIU',
                            'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
                            'CO_UF_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
                            'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN',
                            'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'CO_PROVA_CN',
                            'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TX_RESPOSTAS_CN',
                            'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
                            'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT',
                            'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
                            'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007',
                            'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
                            'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'
                           ]).dropna()



In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 692915 entries, 40 to 3476102
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   NU_INSCRICAO            692915 non-null  int64  
 1   TP_FAIXA_ETARIA         692915 non-null  int64  
 2   TP_SEXO                 692915 non-null  object 
 3   TP_ESTADO_CIVIL         692915 non-null  int64  
 4   TP_COR_RACA             692915 non-null  int64  
 5   TP_ESCOLA               692915 non-null  int64  
 6   SG_UF_ESC               692915 non-null  object 
 7   TP_DEPENDENCIA_ADM_ESC  692915 non-null  float64
 8   TP_LOCALIZACAO_ESC      692915 non-null  float64
 9   NU_NOTA_CN              692915 non-null  float64
 10  NU_NOTA_CH              692915 non-null  float64
 11  NU_NOTA_LC              692915 non-null  float64
 12  NU_NOTA_MT              692915 non-null  float64
 13  NU_NOTA_REDACAO         692915 non-null  float64
 14  Q001              

Neste passo, substituiremos os dados categorizados por strings. A informação foi obtida no arquivo xlsx citado anteriormente.

In [10]:
df_1['TP_DEPENDENCIA_ADM_ESC'] = df_1['TP_DEPENDENCIA_ADM_ESC'].replace({1:'Federal',
                                                                         2:'Estadual',
                                                                         3: 'Municipal',
                                                                         4: 'Privada'})

df_1['TP_ESCOLA'] = df_1['TP_ESCOLA'].replace({1:'Não Respondeu',
                                               2: 'Pública',
                                               3: 'Privada'})

df_1['TP_LOCALIZACAO_ESC'] = df_1['TP_LOCALIZACAO_ESC'].replace({1:'Urbana',
                                                                 2:'Rural'})

df_1['TP_ESTADO_CIVIL'] = df_1['TP_ESTADO_CIVIL'].replace({0: 'Não Informado',
                                                           1:'Solteiro',
                                                           2:'Casado',
                                                           3: 'Divorciado',
                                                           4: 'Viúvo'})

df_1['TP_COR_RACA'] = df_1['TP_COR_RACA'].replace({0: 'Não Declarado',
                                                   1:'Branca',
                                                   2:'Preta',
                                                   3: 'Parda',
                                                   4: 'amarela',
                                                   5: 'indígena',
                                                   6: 'Não dispõe da informação'})

df_1['TP_FAIXA_ETARIA'] = df_1['TP_FAIXA_ETARIA'].replace({1:'menos de 17 anos',
                                                           2:'17 anos',
                                                           3: '18 anos',
                                                           4: '19 anos',
                                                           5: '20 anos',
                                                           6: '21 anos',
                                                           7: '22 anos',
                                                           8: '23 anos',
                                                           9: '24 anos',
                                                          10: '25 anos',
                                                          11: 'entre 26 e 30 anos',
                                                          12: 'entre 31 e 35 anos',
                                                          13: 'entre 36 e 40 anos',
                                                          14: 'entre 41 e 45 anos',
                                                          15: 'entre 46 e 50 anos',
                                                          16: 'entre 51 e 55 anos',
                                                          17: 'entre 56 e 60 anos',
                                                          18: 'entre 61 e 65 anos',
                                                          19: 'entre 66 e 70 anos',
                                                          20: 'mais de 70 anos anos'})

df_1['Q001'] = df_1['Q001'].replace({'A':'Nunca estudou',
                                'B':'Não completou a 4ª série/5º ano do Ensino Fundamental',
                                'C':'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 
                                'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio',
                                'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
                                'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
                                'G': 'Completou a Pós-graduação',
                                'H': 'Não sei'})

df_1['Q002'] = df_1['Q002'].replace({'A':'Nunca estudou',
                                'B':'Não completou a 4ª série/5º ano do Ensino Fundamental',
                                'C':'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 
                                'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio',
                                'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
                                'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
                                'G': 'Completou a Pós-graduação',
                                'H': 'Não sei'})

In [20]:
df_1.head(5)

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q001,Q002
40,210056717205,20 anos,F,Solteiro,Parda,Pública,CE,Estadual,Urbana,436.7,420.0,512.6,430.4,560.0,"Completou a 4ª série/5º ano, mas não completou...",Completou a 8ª série/9º ano do Ensino Fundamen...
55,210056012033,18 anos,F,Solteiro,Branca,Pública,SP,Privada,Urbana,819.5,684.9,657.8,905.5,900.0,Completou a 8ª série/9º ano do Ensino Fundamen...,"Completou a Faculdade, mas não completou a Pós..."
57,210056292241,17 anos,M,Solteiro,Parda,Pública,GO,Estadual,Urbana,442.4,579.2,582.7,551.9,740.0,"Completou a 4ª série/5º ano, mas não completou...",Completou a 8ª série/9º ano do Ensino Fundamen...
59,210054772114,18 anos,F,Solteiro,Parda,Pública,PA,Estadual,Urbana,461.4,485.9,448.9,433.6,520.0,"Completou o Ensino Médio, mas não completou a ...","Completou o Ensino Médio, mas não completou a ..."
67,210055580862,18 anos,M,Solteiro,Branca,Pública,RJ,Estadual,Urbana,427.8,485.0,557.5,438.0,540.0,Completou a 8ª série/9º ano do Ensino Fundamen...,"Completou o Ensino Médio, mas não completou a ..."


In [12]:
df_1.groupby('TP_SEXO').agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'})

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
TP_SEXO,,,,,
F,489.514536,525.962099,519.709732,531.132546,649.804758
M,507.945640,533.708950,518.950502,570.382728,614.793090


In [13]:
df_1.groupby('TP_SEXO').agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,2.000000,2.000000,2.000000,2.000000,2.000000
mean,498.730088,529.835524,519.330117,550.757637,632.298924
std,13.032759,5.477850,0.536857,27.754070,24.756988
min,489.514536,525.962099,518.950502,531.132546,614.793090
25%,494.122312,527.898812,519.140310,540.945092,623.546007
50%,498.730088,529.835524,519.330117,550.757637,632.298924
75%,503.337864,531.772237,519.519925,560.570183,641.051841
max,507.945640,533.708950,519.709732,570.382728,649.804758


In [14]:
df_1.groupby('SG_UF_ESC').agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'})

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
SG_UF_ESC,,,,,
AC,478.002688,506.567010,494.191172,510.761002,610.607368
AL,487.516653,513.408541,501.854346,530.288883,621.740216
AM,471.947725,495.102392,486.058735,494.519225,543.135923
AP,474.361635,505.973217,492.843200,496.912800,598.024348
BA,493.000551,524.054066,512.205453,534.129879,639.966445
CE,474.529790,495.162560,483.021857,508.235091,554.733670
DF,510.462786,544.923512,541.388954,559.605428,656.447911
ES,504.974941,540.527540,525.876101,562.832191,664.592236
GO,492.354807,522.657711,511.045167,538.066551,628.924380


In [15]:
df_1.groupby('SG_UF_ESC').agg({'NU_NOTA_CN':'max', 'NU_NOTA_CH':'max', 'NU_NOTA_LC':'max',
                                        'NU_NOTA_MT':'max', 'NU_NOTA_REDACAO':'max'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,808.177778,794.007407,742.140741,962.192593,984.444444
std,39.929104,29.363045,25.739094,31.009650,8.473185
min,723.200000,720.200000,691.700000,885.400000,980.000000
25%,777.250000,782.050000,724.350000,962.600000,980.000000
50%,806.100000,799.800000,742.000000,977.000000,980.000000
75%,830.250000,817.600000,760.750000,983.200000,980.000000
max,875.300000,839.200000,801.000000,985.700000,1000.000000


In [16]:
df_1.groupby(['TP_COR_RACA']).agg({'NU_NOTA_CN':'max', 'NU_NOTA_CH':'max', 'NU_NOTA_LC':'max',
                                        'NU_NOTA_MT':'max', 'NU_NOTA_REDACAO':'max'})

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
TP_COR_RACA,,,,,
Branca,875.3,829.1,784.2,985.7,1000.0
Não Declarado,868.7,796.9,739.6,981.7,980.0
Parda,826.4,839.2,801.0,985.7,1000.0
Preta,783.6,803.9,749.7,953.7,980.0
amarela,850.3,781.5,737.0,985.7,980.0
indígena,734.5,724.4,730.5,855.0,960.0


In [17]:
df_1.groupby(['TP_COR_RACA']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,487.801061,516.092567,505.108596,530.065931,599.810429
std,18.565727,22.325623,23.008379,33.463230,51.071349
min,461.554101,481.758858,469.395899,483.622867,524.048556
25%,478.840262,510.301130,500.088082,514.632108,585.576575
50%,485.832702,513.792442,503.075203,524.684991,592.480721
75%,498.334299,524.862919,512.741808,549.193359,619.301550
max,514.575913,549.521219,540.048885,578.758775,678.335557


In [18]:
df_1.groupby(['TP_DEPENDENCIA_ADM_ESC']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'})

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
TP_DEPENDENCIA_ADM_ESC,,,,,
Estadual,474.778374,504.351583,496.62126,510.532153,568.036865
Federal,530.840370,565.306578,556.55353,603.398274,724.567396
Municipal,484.312171,515.930990,508.60090,527.347628,586.042244
Privada,538.403070,574.084177,559.04706,615.989949,754.414234


In [19]:
df_1.groupby(['TP_DEPENDENCIA_ADM_ESC']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,507.083496,539.918332,530.205687,564.317001,658.265185
std,32.184152,34.891518,32.252722,53.094205,94.864686
min,474.778374,504.351583,496.621260,510.532153,568.036865
25%,481.928722,513.036138,505.605990,523.143759,581.540899
50%,507.576270,540.618784,532.577215,565.372951,655.304820
75%,532.731045,567.500978,557.176912,606.546193,732.029105
max,538.403070,574.084177,559.047060,615.989949,754.414234


In [28]:
df_1.groupby(['Q001']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).sort_values(by= 'NU_NOTA_REDACAO', 
                                                                                                    ascending=False)

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
Q001,,,,,
Completou a Pós-graduação,546.607281,580.393946,566.095218,630.480338,750.400846
"Completou a Faculdade, mas não completou a Pós-graduação",533.044977,568.557391,557.314258,609.502194,723.644085
"Completou o Ensino Médio, mas não completou a Faculdade",499.272292,533.749575,525.935417,553.386896,650.063510
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",484.936886,515.916699,507.358947,527.399756,607.398857
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",479.748459,509.853279,499.802594,517.821917,590.885024
Não completou a 4ª série/5º ano do Ensino Fundamental,469.656432,495.476432,482.831489,497.965063,556.278263
Não sei,474.790555,502.747719,494.077907,507.028305,554.672162
Nunca estudou,458.696230,478.584495,462.163572,473.904307,499.893874


In [22]:
df_1.groupby(['Q001']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,493.344139,523.159942,511.947425,539.686097,616.654578
std,31.189757,35.553456,35.856534,54.862178,86.454691
min,458.696230,478.584495,462.163572,473.904307,499.893874
25%,473.507025,500.929897,491.266303,504.762495,555.876738
50%,482.342673,512.884989,503.580771,522.610836,599.141941
75%,507.715463,542.451529,533.780127,567.415721,668.458654
max,546.607281,580.393946,566.095218,630.480338,750.400846


In [29]:
df_1.groupby(['Q002']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).sort_values(by= 'NU_NOTA_REDACAO', 
                                                                                                    ascending=False)

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
Q002,,,,,
Completou a Pós-graduação,533.487540,567.464227,555.121446,608.930371,727.584479
"Completou a Faculdade, mas não completou a Pós-graduação",524.735588,559.689431,548.731356,596.153677,706.425373
"Completou o Ensino Médio, mas não completou a Faculdade",493.450654,527.144508,519.588906,543.029683,633.235050
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",477.662477,507.427459,498.715080,514.163805,582.875540
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",472.433785,500.880852,489.859923,503.599636,564.171372
Não completou a 4ª série/5º ano do Ensino Fundamental,465.011992,488.905516,474.843773,487.127172,531.210956
Não sei,467.846735,489.343718,477.511371,494.747418,512.120183
Nunca estudou,454.768812,473.342992,454.410462,465.544440,472.291091


In [24]:
df_1.groupby(['Q002']).agg({'NU_NOTA_CN':'mean', 'NU_NOTA_CH':'mean', 'NU_NOTA_LC':'mean',
                                        'NU_NOTA_MT':'mean', 'NU_NOTA_REDACAO':'mean'}).describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,486.174698,514.274838,502.347790,526.662025,591.239256
std,28.809129,34.276375,36.004364,51.920345,91.367406
min,454.768812,473.342992,454.410462,465.544440,472.291091
25%,467.138049,489.234168,476.844471,492.842357,526.438263
50%,475.048131,504.154155,494.287502,508.881720,573.523456
75%,501.271887,535.280739,526.874518,556.310681,651.532631
max,533.487540,567.464227,555.121446,608.930371,727.584479
